In [1]:
import pandas as pd

In [10]:
df = pd.read_csv(
    f"../data/2023/full.csv",
    dtype={
        "code_commune": "str",
        "code_departement": "str",
        "ancien_nom_commune": "str",
        "numero_volume": "str",
        "lot1_numero": "str",
        "lot2_numero": "str",
        "lot3_numero": "str",
        "lot4_numero": "str",
        "lot5_numero": "str",
        "code_nature_culture_speciale": "str",
        "nature_culture_speciale": "str",
    }
)

In [12]:
df.columns

Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'ancien_code_commune',
       'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle',
       'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude'],
      dtype='object')

In [ ]:
import plotly.express as px

# Create scatter mapbox plot
fig = px.scatter_mapbox(
    df.sample(1000), 
    lat='latitude', 
    lon='longitude',
)

# Use open-street-map style
fig.update_layout(mapbox_style="open-street-map")

# save the plot to a file
fig.write_html(f"../data/{year}/map.html")

In [17]:
import h2o
from h2o.automl import H2OAutoML

# Initialize H2O
h2o.init()

# Convert the pandas dataframe to H2O frame
h2o_frame = h2o.H2OFrame(df)

# Define target and predictors
target = 'valeur_fonciere'
predictors = df.columns.tolist()
predictors.remove(target)

# Split the data into training and testing sets
train, test = h2o_frame.split_frame(ratios=[0.8])

# Run AutoML for regression
aml = H2OAutoML(max_runtime_secs=300)  # Here, 300 seconds is just an example. Adjust it to your needs.
aml.train(x=predictors, y=target, training_frame=train)

# View the AutoML Leaderboard
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))

# Predict with the leader model
preds = aml.leader.predict(test)

# Shutdown H2O instance
h2o.shutdown(prompt=False)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "20.0.1" 2023-04-18; Java(TM) SE Runtime Environment (build 20.0.1+9-29); Java HotSpot(TM) 64-Bit Server VM (build 20.0.1+9-29, mixed mode, sharing)
  Starting server from /Users/lukoadmin/.local/share/virtualenvs/real-estate-analysis-0TLcW-3l/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/fj/ztykb2tx7cg16l9z3q0_c91w0000gp/T/tmp33n4zz9d
  JVM stdout: /var/folders/fj/ztykb2tx7cg16l9z3q0_c91w0000gp/T/tmp33n4zz9d/h2o_lukoadmin_started_from_python.out
  JVM stderr: /var/folders/fj/ztykb2tx7cg16l9z3q0_c91w0000gp/T/tmp33n4zz9d/h2o_lukoadmin_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_lukoadmin_as5l3y
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


TypeError: H2OAutoML got an unexpected keyword argument 'task'